In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import os
import glob

dataset_folder = os.path.abspath("./individual_npzs/{0}/*.npz")

## Parameters

In [2]:
learning_rate = 0.01
batch_size = 64
dropout = 0.75
max_pool = 2
padding = 'same'
strides = 1
activation = 'relu'
input_size = 60000
output_size = 4
epochs = 30

validation_session = 4
test_session = 5

label_dictionary = {'neu': 0, 'hap': 1, 'sad': 2, 'ang': 3}

## Conv params

In [19]:
first_layer_params = {'filters': 16, 'kernel_size': [10, 15], 'strides': [1, 1, 1, 1]}
second_layer_params =  {'filters': 24, 'kernel_size': [8, 10], 'strides': [1, 1, 1, 1]}
third_layer_params =  {'filters': 32, 'kernel_size': [5, 8], 'strides': [1, 1, 1, 1]}

## Load dataset: 3 sessions for training, 1 for validation, 1 for test

In [15]:
train_dataset = []
validation_dataset = []
test_dataset = []

train_labels = []
validation_labels = []
test_labels = []

session_string = 'session{0}'

for i in range(1, 6):
    formatted = session_string.format(i)
    for spectrogram in glob.glob(dataset_folder.format(formatted)):
        loaded_spec = np.load(spectrogram)
        for x in loaded_spec['spectrograms']:
            if i != validation_session and i != test_session:
                train_dataset.append(x) 
            elif i == validation_session:
                validation_dataset.append(x)
            elif i == test_session:
                test_dataset.append(x)
        for x in loaded_spec['labels']:
            if i != validation_session and i != test_session:
                train_labels.append(x) 
            elif i == validation_session:
                validation_labels.append(x)
            elif i == test_session:
                test_labels.append(x)
        
train_dataset = np.asarray(train_dataset)
train_labels = np.asarray(train_labels)

validation_dataset = np.asarray(validation_dataset)
validation_labels = np.asarray(validation_labels)

test_dataset = np.asarray(test_dataset)
test_labels = np.asarray(test_labels)

In [16]:
train_data = np.zeros([len(train_dataset), train_dataset[0].shape[0], train_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(train_dataset)):
    train_data[data,:,:] = train_dataset[data]
    
validation_data = np.zeros([len(validation_dataset), validation_dataset[0].shape[0], validation_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(validation_dataset)):
    train_data[data,:,:] = validation_dataset[data]
    
test_data = np.zeros([len(test_dataset), test_dataset[0].shape[0], test_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(test_dataset)):
    train_data[data,:,:] = test_dataset[data]

## Convolutional

In [21]:
def conv2d(to_process, weights, biases, strides=1):
    conv_out = tf.nn.conv2d(to_process, weights, strides=[1, strides, strides, 1], padding='SAME')
    bias_out = tf.nn.bias_add(conv_out, biases)
    relu_out = tf.nn.relu(bias_out)
    return relu_out

def maxpool2d(to_pool, pool_size=2):
    maxpool_out = tf.nn.max_pool(to_pool, ksize=[1, pool_size, pool_size, 1], strides=[1, pool_size, pool_size, 1], padding='SAME')
    return maxpool_out

def nn_pipeline(spectrogram, weights, biases):
    
    reshaped_input = tf.reshape(spectrogam, shape=[-1, 200, 300, 1])
    
    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, k=2)
    
    second_layer_out = conv2d(first_maxpool_out, weights['second_layer_weights'], biases['second_layer_biases'])
    second_maxpool_out = maxpool2d(second_layer_out, k=2)
    
    third_layer_out = conv2d(second_maxpool_out, weights['third_layer_weights'], biases['third_layer_biases'])
    first_maxpool_out = maxpool2d(first_layer_out, k=2)

In [20]:
def convnet(x_data, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = x_data
        
        x = tf.reshape(x, shape=[-1, 200, 300, 1])
        
        first_layer_out_1 = tf.layers.conv2d(x, first_layer_params['filters'], 
                                       first_layer_params['kernel_size'], 
                                       first_layer_params['strides'], activation=tf.nn.relu)
        first_layer_out_2 = tf.layers.max_pooling2d(first_layer_out_1, 2, 2)
        
        second_layer_out_1 = tf.layers.conv2d(first_layer_out_2, second_layer_params['filters'], 
                                       second_layer_params['kernel_size'], 
                                       second_layer_params['strides'], activation=tf.nn.relu)
        second_layer_out_2 = tf.layers.max_pooling2d(second_layer_out_1, 2, 2)
        
        third_layer_out_1 = tf.layers.conv2d(second_layer_out_2, third_layer_params['filters'], 
                                       third_layer_params['kernel_size'], 
                                       third_layer_params['strides'], activation=tf.nn.relu)
        third_layer_out_2 = tf.layers.max_pooling2d(third_layer_out_1, 2, 2)
        
        flatten_out = tf.contrib.layers.flatten(third_layer_out_2)

        dense_out = tf.layers.dense(flatten_out, 32 * flatten_out.shape[0] * flatten_out.shape[1])
        dropout_out = tf.layers.dropout(dense_out, rate=dropout, training=is_training)

        prediction = tf.layers.dense(dropout_out, output_size)
    return prediction